# q3(C4.5)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
data = {
    'Income': ['Low', 'Low', 'Medium', 'Medium', 'High', 'High'],
    'Credit': ['Good', 'Bad', 'Good', 'Bad', 'Good', 'Bad'],
    'Loan Approved': ['Yes', 'No', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)
print(df)


   Income Credit Loan Approved
0     Low   Good           Yes
1     Low    Bad            No
2  Medium   Good           Yes
3  Medium    Bad           Yes
4    High   Good           Yes
5    High    Bad            No


In [10]:

def entropy(data):
    p,n = 0,0
    x = data.value_counts("Loan Approved")
    if len(x) > 1:
        if data.iat[0,-1] == "No":
            n,p = x[0],x[1]
        else:
            p,n = x[0],x[1]
    else:
        if data.iat[0,-1] == "No":
            n = x[0]
        else:
            p = x[0]
    p_ratio = p/(p+n)
    n_ratio = 1 - p_ratio
    entropy_p = -p_ratio*math.log2(p_ratio) if p_ratio != 0 else 0
    entropy_n = - n_ratio*math.log2(n_ratio) if n_ratio !=0 else 0
    return entropy_p + entropy_n

def gain_ratio(data,feature):
    unique_values = data[feature].unique()
    feature_weights = [len(data[data[feature] == val]) for val in unique_values]
    entropies = [entropy(data[data[feature] == val]) for val in unique_values]
    info_gain_feature = sum(feature_weights[i] / sum(feature_weights) * entropies[i] for i in range(len(unique_values)))
    info_gain = entropy(data) - info_gain_feature
    split_info = sum(-1*feature_weights[i] / sum(feature_weights) * math.log2(feature_weights[i] / sum(feature_weights)) for i in range(len(unique_values)))
    gain_ratio = info_gain/split_info
    return gain_ratio



entropy(df)
print(gain_ratio(df,"Income"))
print(gain_ratio(df,"Credit"))


0.15876032857139014
0.4591479170272448


/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_10888/2588909633.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

class Node:
    def __init__(self, data, parent_feature=None, parent_feature_value=None):
        self.data = data  
        self.parent_feature = parent_feature  
        self.parent_feature_value = parent_feature_value  
        self.children = {} 

def c45_decision_tree(data, features, target, parent_feature=None, parent_feature_value=None):
    node = Node(data, parent_feature, parent_feature_value)    
    if len(data['Loan Approved'].unique()) == 1:
        return data['Loan Approved'].values[0]    
    if len(features) == 0:
        return data['Loan Approved'].mode().values[0]    
    gain_ratios = {feature: gain_ratio(data, feature) for feature in features}
    best_feature = max(gain_ratios, key=gain_ratios.get)    
    for value in data[best_feature].unique():
        child_data = data[data[best_feature] == value].drop([best_feature], axis=1)
        node.children[value] = c45_decision_tree(child_data, features.drop(best_feature), target, best_feature, value)
    return node

features = df.columns[:-1]
target = 'Loan Approved'
decision_tree = c45_decision_tree(df, features, target)



/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_10888/2588909633.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer

In [14]:
def print_c45_decision_tree(node, indent=""):
    if isinstance(node, Node):
        if node.parent_feature is not None:
            print(indent + "Feature: " + node.parent_feature)
            print(indent + "Parent Feature Value: " + str(node.parent_feature_value))
        
        if len(node.children) == 0:
            print(indent + "Loan Approved: " + str(node.data['Loan Approved'].values[0]))
        else:
            for value, child_node in node.children.items():
                print(indent + "Value: " + str(value))
                print_c45_decision_tree(child_node, indent + "  ")
    else:
        print(indent + "Approved?: " + str(node))

print_c45_decision_tree(decision_tree)

Value: Good
  Approved?: Yes
Value: Bad
  Feature: Credit
  Parent Feature Value: Bad
  Value: Low
    Approved?: No
  Value: Medium
    Approved?: Yes
  Value: High
    Approved?: No


# q3(CART)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 

print(df)

def gini(data):
    p,n = 0,0
    x = data.value_counts("Loan Approved")
    if len(x) > 1:
        if data.iat[0,-1] == "No":
            n,p = x[0],x[1]
        else:
            p,n = x[0],x[1]
    else:
        if data.iat[0,-1] == "No":
            n = x[0]
        else:
            p = x[0]
    p_ratio = p/(p+n)
    n_ratio = 1 - p_ratio
    gini_p = (p_ratio)**2
    gini_n = (n_ratio)**2
    return 1 - gini_p - gini_n

def gini_feature(data,feature):
    unique_values = data[feature].unique()
    feature_weights = [len(data[data[feature] == val]) for val in unique_values]
    ginies = [gini(data[data[feature] == val]) for val in unique_values]
    gini_feature_val = sum(feature_weights[i] / sum(feature_weights) * ginies[i] for i in range(len(unique_values)))
    return gini_feature_val

print(entropy(df))
print(gini_feature(df,"Income"))
print(gini_feature(df,"Credit"))

   Income Credit Loan Approved
0     Low   Good           Yes
1     Low    Bad            No
2  Medium   Good           Yes
3  Medium    Bad           Yes
4    High   Good           Yes
5    High    Bad            No
0.9182958340544896
0.3333333333333333
0.22222222222222215


/tmp/ipykernel_10888/2588909633.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_10888/2928221754.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p,n = x[0],x[1]
/tmp/ipykernel_10888/2928221754.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p = x[0]
/tmp/ipykernel_10888/2928221754.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integ

In [1]:
def build_cart_decision_tree(data, features, target, parent_feature=None, parent_feature_value=None):
    node = Node(data, parent_feature, parent_feature_value)
    if len(data['Loan Approved'].unique()) == 1:
        return data['Loan Approved'].values[0]
    if len(features) == 0:
        return data['Loan Approved'].mode().values[0]
    gini_indices = {feature: gini_feature(data, feature) for feature in features}
    best_feature = min(gini_indices, key=gini_indices.get)
    for value in data[best_feature].unique():
        child_data = data[data[best_feature] == value].drop([best_feature], axis=1)
        node.children[value] = build_cart_decision_tree(child_data, features.drop(best_feature), target, best_feature, value)
    return node
features = df.columns[:-1]
target = 'Loan Approved'

decision_tree = build_cart_decision_tree(df, features, target)
def print_cart_decision_tree(node, indent=""):
    if isinstance(node, Node):
        if node.parent_feature is not None:
            print(indent + "Feature: " + node.parent_feature)
            print(indent + "Parent Feature Value: " + str(node.parent_feature_value))

        if len(node.children) == 0:
            print(indent + "Loan Approved: " + str(node.data['Loan Approved'].values[0]))
        else:
            for value, child_node in node.children.items():
                print(indent + "Value: " + str(value))
                print_cart_decision_tree(child_node, indent + "  ")
    else:
        print(indent + "Approved?: " + str(node))
        
print_cart_decision_tree(decision_tree)

NameError: name 'df' is not defined